In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.layers import Dense, Add
from tensorflow.keras.models import Sequential

import os



In [2]:
path = '../input/colorectal-histology-mnist/Kather_texture_2016_image_tiles_5000/Kather_texture_2016_image_tiles_5000'
IDG = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0.1,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=None,
    shear_range=0.1,
    zoom_range=0.4,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.1,
    dtype=None,
)
train_dg =IDG.flow_from_directory(
    path,
    color_mode=('rgb'),
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    subset=None,
    interpolation="nearest",
)

Found 5000 images belonging to 8 classes.


In [3]:
import requests
url = 'https://github.com/Fsunroo/RSNA-BoneAge/releases/download/v1.0/output.hdf5'
r = requests.get(url)
with open('RSNA_Xception_1.hdf5','wb') as f:
    f.write(r.content)
source_model = tf.keras.models.load_model('RSNA_Xception_1.hdf5')
source_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 8, 8, 2048)        20861480  
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 20,881,981
Trainable params: 20,827,453
Non-trainable params: 54,528
_________________________________________________________________


In [4]:
model = Sequential()
for layer in source_model.layers[:-1]: # go through until last layer
    model.add(layer)
model.add(Dense(8, activation='softmax',name='dense_2'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 8, 8, 2048)        20861480  
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
Total params: 20,882,058
Trainable params: 20,827,530
Non-trainable params: 54,528
_________________________________________________________________


In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = 'accuracy')

In [6]:
history = model.fit(
    train_dg,  epochs=20, verbose=1,
    class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False,
    shuffle=True, initial_epoch=0
)

Epoch 1/20
157/157 [==============================] - 130s 775ms/step - loss: 1.6936 - accuracy: 0.3209
Epoch 2/20
157/157 [==============================] - 108s 682ms/step - loss: 0.8943 - accuracy: 0.6402
Epoch 3/20
157/157 [==============================] - 107s 683ms/step - loss: 0.7867 - accuracy: 0.6912
Epoch 4/20
157/157 [==============================] - 107s 681ms/step - loss: 0.6104 - accuracy: 0.7868
Epoch 5/20
157/157 [==============================] - 108s 686ms/step - loss: 0.4671 - accuracy: 0.8371
Epoch 6/20
157/157 [==============================] - 108s 687ms/step - loss: 0.4759 - accuracy: 0.8439
Epoch 7/20
157/157 [==============================] - 109s 692ms/step - loss: 0.4130 - accuracy: 0.8689
Epoch 8/20
157/157 [==============================] - 108s 686ms/step - loss: 0.3582 - accuracy: 0.8789
Epoch 9/20
157/157 [==============================] - 109s 691ms/step - loss: 0.3795 - accuracy: 0.8794
Epoch 10/20
157/157 [==============================] - 110s 698m

In [7]:
model.save('Xception on CRC-2.hdf5')

<a href="./RSNA_Xception_1.hdf5"> Download File </a>
